<a href="https://colab.research.google.com/github/palexey2006/Ml-Engineering/blob/main/CNN_models/Resnet_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import random_split
import numpy as np
import sys
class block(nn.Module):
    def __init__(self,in_channels, intermediate_channels,identity_downsample=None, stride=1):
        super().__init__()
        self.expansion = 4
        self.cn1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.cn2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3,stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.cn3 = nn.Conv2d(intermediate_channels, intermediate_channels * self.expansion, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.downsample = identity_downsample
        self.stride = stride
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.bn1(self.cn1(x)))
        x = self.relu(self.bn2(self.cn2(x)))
        x = self.bn3(self.cn3(x))

        if self.downsample is not None:
            identity = self.downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet,self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.create_layer(block,layers[0],64,1)
        self.layer2 = self.create_layer(block,layers[1],128,2)
        self.layer3 = self.create_layer(block,layers[2],256,2)
        self.layer4 = self.create_layer(block,layers[3],512,2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self,x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

        #creating a layer

    def create_layer(self,block, num_residual_blocks, intermediate_channels,stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels,
                                                          intermediate_channels * 4,
                                                          kernel_size=1,
                                                          stride=stride),
                                                          nn.BatchNorm2d(intermediate_channels * 4))
        layers.append(block(self.in_channels,intermediate_channels,identity_downsample,stride))
        self.in_channels = intermediate_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels,intermediate_channels))

        return nn.Sequential(*layers)



def Resnet50(image_channels=3, num_classes=1000):
    return ResNet(block=block, layers=[3,4,6,3],image_channels=image_channels, num_classes=num_classes)

def Resnet101(image_channels=3, num_classes=1000):
    return ResNet(block=block,layers=[3,4,23,3],image_channels=image_channels,num_classes=num_classes)

def Resnet152(image_channels=3, num_classes=1000):
    return ResNet(block=block,layers=[3,8,36,3],image_channels=image_channels, num_classes=num_classes)

model = Resnet50()
test = torch.randn((1,3,224,224))
print(model(test).shape)


torch.Size([1, 1000])
